In [1]:
#Dependencies and set up
import pandas as pd
from pathlib import Path

In [2]:
# Loading and Reading each file
schoolfile = Path("Resources/schools_complete.csv")
schooldf = pd.read_csv(schoolfile)
studentfile = Path ("Resources/students_complete.csv")
studentdf = pd.read_csv(studentfile)

In [3]:
#Merging Data on school name
complete_schooldf = pd.merge(studentdf, schooldf, how="left", on=["school_name", "school_name"])
complete_schooldf.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# DISTRICT SUMMARY

In [4]:
#Total Number of Unique Schools
schoolcount = complete_schooldf["school_name"].nunique()
schoolcount


15

In [5]:
#Total Students
studentcount = complete_schooldf["student_name"].count()
studentcount

39170

In [6]:
#Total Budget -- where type == charter and district
totalbudget = schooldf["budget"].sum()
totalbudget

24649428

In [7]:
#Average Math Score
districtmath = complete_schooldf["math_score"].mean()
districtmath

78.98537145774827

In [8]:
#Average Reading Score
districtreading = complete_schooldf["reading_score"].mean()
districtreading

81.87784018381414

In [9]:
#Percent of students passing math
passmathcount = complete_schooldf[(complete_schooldf["math_score"] >= 70)].count()["student_name"]
passmathpercent = passmathcount / float(studentcount) * 100
passmathpercent

74.9808526933878

In [10]:
#Percent of students passing reading
passreadingcount = complete_schooldf[(complete_schooldf["reading_score"] >= 70)].count()["student_name"]
passreadingpercent = passreadingcount / float(studentcount) * 100
passreadingpercent

85.80546336482001

In [11]:
#Percent of students who passed math and reading
percent_math_reading_count = complete_schooldf[(complete_schooldf["math_score"]>= 70) & (complete_schooldf["reading_score"] >=70)].count()["student_name"]
overall_passing_rate = percent_math_reading_count / float(studentcount) * 100
overall_passing_rate

65.17232575950983

In [12]:
#District key metrics DataFrame
districtmetrics = pd.DataFrame([[schoolcount, studentcount, totalbudget, districtmath, districtreading, passmathpercent, passreadingpercent, overall_passing_rate]],
                               columns=["Total Schools", "Total Students", "Total Budget", 
                                "Average Math Score", "Average Reading Score", 
                                "% Passing Math", "% Passing Reading", "% Overall Passing"])

#Formatting (commas, dollar signs, percent signs, decimals)
districtmetrics["Total Students"] = districtmetrics["Total Students"].map("{:,}".format)
districtmetrics["Total Budget"] = districtmetrics["Total Budget"].map("${:,.2f}".format)
districtmetrics["Average Math Score"] = districtmetrics["Average Math Score"].map("{:.2f}".format)
districtmetrics["Average Reading Score"] = districtmetrics["Average Reading Score"].map("{:.2f}".format)
districtmetrics["% Passing Math"] = districtmetrics["% Passing Math"].map("{:.2f}%".format)
districtmetrics["% Passing Reading"] = districtmetrics["% Passing Reading"].map("{:.2f}%".format)
districtmetrics["% Overall Passing"] = districtmetrics["% Overall Passing"].map("{:.2f}%".format)

#Display Distric DataFrame
districtmetrics

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


# SCHOOL SUMMARY

In [15]:
#Set index to school and sort alphabetically
per_schooldf = schooldf.set_index(['school_name']).sort_values('school_name')

#Identify type of school for each
schooltypes = schooldf.set_index(['school_name'])['type']

#Size of School
student_count = per_schooldf['size']

#Budget for school and budget:student ratio
total_budget = per_schooldf['budget']
per_student_budget = total_budget/student_count

#Average math and reading scores for each school
avg_math_score = complete_schooldf.groupby('school_name')['math_score'].mean()
avg_reading_score = complete_schooldf.groupby('school_name')['reading_score'].mean()

#Percent passing math and percent passing reading for each school
mathpass = complete_schooldf[complete_schooldf['math_score']>= 70]
mathpasscount = mathpass.groupby('school_name')['student_name'].count()
percentschool_math_pass = mathpasscount / student_count * 100

readingpass = complete_schooldf[complete_schooldf['reading_score']>= 70]
readingpasscount = readingpass.groupby('school_name')['student_name'].count()
percentschool_reading_pass = readingpasscount / student_count * 100

#Percent passing both math and reading
overallpass = complete_schooldf[(complete_schooldf['math_score']>= 70) & (complete_schooldf['reading_score']>= 70)]
overallpasscount = overallpass.groupby('school_name')['student_name'].count()
percentschool_overall_pass = overallpasscount / student_count * 100


#Complete DataFrame with calculations
per_schooldf = pd.DataFrame({'School Type': schooltypes,
                             'Total Students': student_count,
                             'Total School Budget': total_budget,
                             'Per Student Budget': per_student_budget,
                             'Average Math Score': avg_math_score,
                             'Average Reading Score': avg_reading_score,
                             '% Passing Math': percentschool_math_pass,
                             '% Passing Reading': percentschool_reading_pass,
                             '% Overall Passing':percentschool_overall_pass})
per_schooldf.index.name= None
per_schooldf

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [16]:
# Formatting

#per_schooldf['Total School Budget'] = per_schooldf['Total School Budget'].map("${:,.2f}".format)
#per_schooldf['Per Student Budget'] = per_schooldf['Per Student Budget'].astype(float).map("${:,.2f}".format)

#per_schooldf['% Passing Math'] = per_schooldf['% Passing Math'].map("{:.2f}%".format)
#per_schooldf['% Passing Reading'] = per_schooldf['% Passing Reading'].map("{:.2f}%".format)
#per_schooldf['% Overall Passing'] = per_schooldf['% Overall Passing'].map("{:.2f}%".format)


# High Performing Schools

In [17]:
#Identify the schools with the highest overall passing percentage
top_performers = per_schooldf.sort_values('% Overall Passing', ascending=False)
top_performers.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


# Low Performing Schools

In [18]:
#Identify the schools with the lowest overall passing percentage
low_performers = per_schooldf.sort_values('% Overall Passing')
low_performers.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


# Reading Score by Grade

In [19]:
#Create a new table for each grade by locating desired class from 'grade' column in the complete_schooldf
ninth_graders = complete_schooldf[(complete_schooldf["grade"] == "9th")]
tenth_graders = complete_schooldf[(complete_schooldf["grade"] == "10th")]
eleventh_graders = complete_schooldf[(complete_schooldf["grade"] == "11th")]
twelfth_graders = complete_schooldf[(complete_schooldf["grade"] == "12th")]

#Group each new table by school name and get the mean reading score for each
reading_scores_by_grade = schooldf.set_index('school_name').sort_values('school_name')
reading_scores_by_grade['9th'] = ninth_graders.groupby('school_name')['reading_score'].mean()
reading_scores_by_grade['10th'] = tenth_graders.groupby('school_name')['reading_score'].mean()
reading_scores_by_grade['11th'] = eleventh_graders.groupby('school_name')['reading_score'].mean()
reading_scores_by_grade['12th'] = twelfth_graders.groupby('school_name')['reading_score'].mean()

#Create a new DataFrame, index = schools, columns = grades
reading_scores_by_grade = reading_scores_by_grade[['9th','10th','11th','12th']]
reading_scores_by_grade.index.name = None
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending

In [20]:
# Establish the bins
bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [21]:
#Copy per_schooldf
school_spending = per_schooldf.copy()

#Create a new column for labels - bin each school based on per student spending
school_spending['Spending Ranges (per Student)'] = pd.cut(school_spending['Per Student Budget'], 
                                              bins, labels=labels,)

In [22]:
#Find the mean score and passing rates for each spending range
spending_math_scores = school_spending.groupby(["Spending Ranges (per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending.groupby(["Spending Ranges (per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending.groupby(["Spending Ranges (per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending.groupby(["Spending Ranges (per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending.groupby(["Spending Ranges (per Student)"])["% Overall Passing"].mean()

#Create a summary dataframe to show subject outcomes by spending ranges
spending_summary = school_spending.set_index('Spending Ranges (per Student)')
spending_summary = pd.DataFrame({'Average Math Score': spending_math_scores,
                                 'Average Reading Score': spending_reading_scores,
                                 '% Passing Math': spending_passing_math,
                                 '% Passing Reading': spending_passing_reading,
                                 '% Overall Passing': overall_passing_spending})
spending_summary


/var/folders/9z/b56mdrgn3dx15vhy6pqfhjk00000gn/T/ipykernel_85686/1656652373.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending.groupby(["Spending Ranges (per Student)"])["Average Math Score"].mean()
/var/folders/9z/b56mdrgn3dx15vhy6pqfhjk00000gn/T/ipykernel_85686/1656652373.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending.groupby(["Spending Ranges (per Student)"])["Average Reading Score"].mean()
/var/folders/9z/b56mdrgn3dx15vhy6pqfhjk00000gn/T/ipykernel_85686/1656652373.py:3: FutureWarning: The default of observed=False is de

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


# Scores by School Size


In [23]:
#Establish bins and labels
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [24]:
#Create new column for school size and bin each school based on total number of students 
per_schooldf["School Size"] = pd.cut(per_schooldf['Total Students'], 
                                              size_bins, labels=labels,)

In [25]:
# Calculate averages for the desired columns.
size_math_scores = per_schooldf.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_schooldf.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_schooldf.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_schooldf.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_schooldf.groupby(["School Size"])["% Overall Passing"].mean()

/var/folders/9z/b56mdrgn3dx15vhy6pqfhjk00000gn/T/ipykernel_85686/4266878381.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = per_schooldf.groupby(["School Size"])["Average Math Score"].mean()
/var/folders/9z/b56mdrgn3dx15vhy6pqfhjk00000gn/T/ipykernel_85686/4266878381.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_schooldf.groupby(["School Size"])["Average Reading Score"].mean()
/var/folders/9z/b56mdrgn3dx15vhy6pqfhjk00000gn/T/ipykernel_85686/4266878381.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future v

In [26]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = per_schooldf.set_index('School Size')

size_math_scores = per_schooldf.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_schooldf.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_schooldf.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_schooldf.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_schooldf.groupby(["School Size"])["% Overall Passing"].mean()

#size_summary['Average Math Score'] = size_math_scores
size_summary = pd.DataFrame({'Average Math Score': size_math_scores, 
                             'Average Reading Score': size_reading_scores,
                             '% Passing Math': size_passing_math,
                             '% Passing Reading': size_passing_reading,
                             '% Overall Passing': size_overall_passing})

# Display results
size_summary

/var/folders/9z/b56mdrgn3dx15vhy6pqfhjk00000gn/T/ipykernel_85686/3660012073.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = per_schooldf.groupby(["School Size"])["Average Math Score"].mean()
/var/folders/9z/b56mdrgn3dx15vhy6pqfhjk00000gn/T/ipykernel_85686/3660012073.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_schooldf.groupby(["School Size"])["Average Reading Score"].mean()
/var/folders/9z/b56mdrgn3dx15vhy6pqfhjk00000gn/T/ipykernel_85686/3660012073.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future v

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


# Scores by School Type

In [27]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_schooldf.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_schooldf.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_schooldf.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_schooldf.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_schooldf.groupby(["School Type"])["% Overall Passing"].mean()

type_summary = per_schooldf.set_index('School Type')

type_summary = pd.DataFrame({ 'Average Math Score': average_math_score_by_type,
                             'Average Reading Score': average_reading_score_by_type,
                             '% Passing Math': average_percent_passing_math_by_type,
                             '% Passing Reading': average_percent_passing_reading_by_type,
                             '% Overall Passing': average_percent_overall_passing_by_type})
type_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


In [ ]:
# Formatting - Per School Data

per_schooldf['Total School Budget'] = per_schooldf['Total School Budget'].map("${:,.2f}".format)
per_schooldf['Per Student Budget'] = per_schooldf['Per Student Budget'].astype(float).map("${:,.2f}".format)

per_schooldf['% Passing Math'] = per_schooldf['% Passing Math'].map("{:.2f}%".format)
per_schooldf['% Passing Reading'] = per_schooldf['% Passing Reading'].map("{:.2f}%".format)
per_schooldf['% Overall Passing'] = per_schooldf['% Overall Passing'].map("{:.2f}%".format)

#Formatting - performace data
top_performers['Total School Budget'] = top_performers['Total School Budget'].map("${:,.2f}".format)
low_performers['Total School Budget'] = low_performers['Total School Budget'].map("${:,.2f}".format)